In [2]:
from geopy.geocoders import Nominatim

In [3]:
geolocator = Nominatim(user_agent="speficy_your_app_name_here")

In [4]:
location = geolocator.reverse("52.509669, 13.376294")

In [5]:
print(location.address)

Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland


In [77]:
import geopy.geocoders

In [78]:
#너무 작게 잡으면 Service Timeout Error 문제가 발생함
geopy.geocoders.options.default_timeout=5

In [79]:
geolocator = Nominatim(user_agent="speficy_your_app_name_here")

In [80]:
print(geolocator.timeout)

5


In [12]:
location = geolocator.reverse("37.484146, 127.011653")

In [13]:
location

Location(남서울농협, 효령로, 서초3동, 서초동, 서초구, 서울, 06720, 대한민국, (37.483955, 127.012007, 0.0))

In [14]:
# calculate distance between two points

from geopy.distance import geodesic

In [15]:
a = (37.484146, 127.011653)

In [29]:
b = (37.485146, 127.011653)

In [33]:
c = (37.484146, 127.012653)

In [35]:
print(geodesic(a,b).meters) #0.001 longitude = 110.9867 meter

110.98674757137965


In [36]:
print(geodesic(a,c).meters) #0.001 latitude = 88.4414 meter

88.44413618387576


In [42]:
long_amp = 0.002
lati_amp = 0.002

In [44]:
start_long = 37.484146
start_lati = 127.011653
for i in range(10):
    location = geolocator.reverse(str(start_long)+", "+str(start_lati))
    print(location)
    #start_long += long_amp
    start_lati += lati_amp

남서울농협, 효령로, 서초3동, 서초동, 서초구, 서울, 06720, 대한민국
남서울농협, 효령로, 서초3동, 서초동, 서초구, 서울, 06720, 대한민국
반포대로10길, 서초3동, 서초동, 서초구, 서울, 06711, 대한민국
반포대로10길, 서초3동, 서초동, 서초구, 서울, 06711, 대한민국
서울남부터미널, 효령로, 서초3동, 서초동, 서초구, 서울, 06720, 대한민국
서울남부터미널, 효령로, 서초3동, 서초동, 서초구, 서울, 06720, 대한민국
한국시티은행, 서초중앙로, 서초1동, 서초동, 서초구, 서울, 06720, 대한민국
NH농협은행, 효령로60길, 서초1동, 서초동, 서초구, 서울, 06720, 대한민국
부일갈비, 서초중앙로2길, 서초1동, 서초동, 서초구, 서울, 06720, 대한민국
남부순환로337길, 서초1동, 서초동, 서초구, 서울, 06720, 대한민국


In [89]:
# longitude (크면 위 작으면 아래), latitude (크면 오른쪽 작으면 왼쪽)
# 각 구에 해당하는 모든 위도 경도를 뽑아내보자(0.002 long, 0.002 lati)간격으로
# 구 영역에 해당하는 latitudes에 대해 최상단 및 최하단의 longitude를 저장하자
def make_gu_list(st_long,st_lati):
    #시작 주소로 어느 구의 list를 생성할건지 저장
    location = geolocator.reverse(str(start_long)+", "+str(start_lati))
    gu = location.address.split(', ')[-4]
    #print(gu)
    res = {}
    point_long = st_long
    point_lati = st_lati
    #Find top longitude
    '''
    while True:
        location = geolocator.reverse(str(point_long)+", "+str(point_lati))
        if location.address.split(', ')[-4] != gu:
            break
        #pos = {'longitude':point_long, 'latitude':point_lati, 'address':location.address}
        #res.append(pos)
        point_long += long_amp
        print(location.address)
        
    res[point_lati] = [point_long]
    point_long = st_long
    while True:
        location = geolocator.reverse(str(point_long)+", "+str(point_lati))
        if location.address.split(', ')[-4] != gu:
            break
        #pos = {'longitude':point_long, 'latitude':point_lati, 'address':location.address}
        #res.append(pos)
        point_long -= long_amp
        #print(pos)
        print(location.address)
    res[point_lati].append(point_long)
    '''
    res[point_lati]=[37.51114599999994, 37.45514600000007]
    print('res[',point_lati,']:',res[point_lati])
    
    def find_extension(t_long,d_long,lati):
        t_p = t_long
        location = geolocator.reverse(str(t_p)+", "+str(lati))
        #print(location.address)
        while location.address.split(', ')[-4] == gu:
            t_p += long_amp
            
            location = geolocator.reverse(str(t_p)+", "+str(lati))

        if location.address.split(', ')[-4] != gu:
            while location.address.split(', ')[-4] != gu:
                t_p -= long_amp
                if t_p < d_long:
                    t_p=-1
                    break
                location = geolocator.reverse(str(t_p)+", "+str(lati))
        #해당 latitude가 구를 벗어난 경우
        if t_p ==-1:
            return (-1,-1)
        
        d_p = d_long
        location = geolocator.reverse(str(d_p)+", "+str(lati))
        #print(location.address)
        while location.address.split(', ')[-4] == gu:
            d_p -= long_amp
            #print('hio')
            location = geolocator.reverse(str(d_p)+", "+str(lati))

        if location.address.split(', ')[-4] != gu:
            while location.address.split(', ')[-4] != gu:
                d_p += long_amp
                location = geolocator.reverse(str(d_p)+", "+str(lati))
        return (t_p,d_p)            
    
    while True:
        t_long = res[point_lati][0]
        d_long = res[point_lati][1]
        point_lati += lati_amp
        res_t_long, res_d_long = find_extension(t_long,d_long,point_lati)
        if res_t_long == -1:
            break
        else:
            res[point_lati]=[res_t_long,res_d_long]
            print(point_lati,":",res[point_lati])
            location = geolocator.reverse(str(res_t_long)+", "+str(point_lati))
            print(location.address)
            location = geolocator.reverse(str(res_d_long)+", "+str(point_lati))            
            print(location.address)
    point_lati = st_lati
    while True:
        t_long = res[point_lati][0]
        d_long = res[point_lati][1]
        point_lati -= lati_amp
        res_t_long, res_d_long = find_extension(t_long,d_long,point_lati)
        if res_t_long == -1:
            break
        else:
            res[point_lati]=[res_t_long,res_d_long]
            print(point_lati,":",res[point_lati])
    return res

In [90]:
res = make_gu_list(start_long,start_lati)

res[ 127.02165300000004 ]: [37.51114599999994, 37.45514600000007]
127.02265300000005 : [37.50814599999995, 37.457146000000066]
강남대로, 논현1동, 논현동, 서초구, 서울, 06609, 대한민국
양재대로2길, LH4,5단지, 양재1동, 우면동, 서초구, 서울, 06768, 대한민국
127.02365300000005 : [37.50614599999995, 37.457146000000066]
논현역, 강남대로, 논현1동, 논현동, 서초구, 서울, 06609, 대한민국
양재대로2길, LH4,5단지, 양재1동, 우면동, 서초구, 서울, 06768, 대한민국
127.02465300000006 : [37.50314599999996, 37.45814600000006]
백암빌딩, 459, 강남대로, 서초4동, 서초동, 서초구, 서울, 06611, 대한민국
양재대로2길, LH4,5단지, 양재1동, 우면동, 서초구, 서울, 06768, 대한민국
127.02565300000006 : [37.50114599999996, 37.45814600000006]
주류성빌딩, 435, 강남대로, 서초4동, 서초동, 서초구, 서울, 06612, 대한민국
양재천 우측 자전거도로, 양재1동, 양재동, 서초구, 서울, 06746, 대한민국
127.02665300000007 : [37.49914599999997, 37.45914600000006]
대승빌딩, 3, 강남대로61길, 서초4동, 서초동, 서초구, 서울, 06615, 대한민국
양재천 우측 자전거도로, 양재1동, 양재동, 서초구, 서울, 06746, 대한민국
127.02765300000007 : [37.496145999999975, 37.45914600000006]
모델하우스, 서초대로78길, 서초2동, 서초동, 서초구, 서울, 06620, 대한민국
고향식물원, 양재대로, 양재1동, 양재동, 서초구, 서울, 06768, 대한민국
127.02865

KeyboardInterrupt: 